## Load the necessary imports

In [1]:
from __future__ import annotations

import sys

sys.path.append("../scripts")

In [2]:
from tqdm import tqdm
from testy import count_black_pixels, convert_slice_stats_to_csv, convert_volume_stats_to_csv, comparison_plot
from wup_analysis import initialize_dataset, DATA_PATH_TEMPLATE_1, update_slice_stats, update_volume_stats
from preprocessing import adaptive_thresholding
import cv2
from matplotlib import pyplot as plt
import numpy as np

## Implementation of helper functions for the image preprocessing

## Main function for the 2D image analysis

In [3]:
def run_analysis(data: list[np.ndarray], dataset: str, save: bool = False) -> None:
    """
    Perform analysis on the given dataset.

    :param data: List of numpy arrays representing the dataset.
    :param dataset: Name of the dataset.
    :param save: Whether to save the analysis results or not. Defaults to False.
    :return: None
    """

    slice_stats = {}
    min_init, max_init = float('inf'), 0
    volume_stats = {
        'min_air_pockets': min_init,
        'max_air_pockets': max_init,
        'min_air_pocket_size': min_init,
        'max_air_pocket_size': max_init,
        'min_black_pixel_count': min_init,
        'max_black_pixel_count': max_init,
        'min_air_pocket_percentage': min_init,
        'max_air_pocket_percentage': max_init,
    }

    for img in tqdm(data, total=len(data)):
        center_x = img.shape[1] // 2
        center_y = img.shape[0] // 2
        thresh = adaptive_thresholding(img, block_size=201, constant_offset=50)
        volume_black_pixel_count = count_black_pixels(thresh)
        inverted_img = cv2.bitwise_not(thresh)

        num_labels, _, stats, centroids = cv2.connectedComponentsWithStats(inverted_img, connectivity=8)

        slice_stats, areas, max_area, min_area = update_slice_stats(slice_stats, num_labels, stats, center_x, center_y, centroids)
        volume_stats = update_volume_stats(volume_stats, num_labels, min_area, max_area, volume_black_pixel_count, areas)

    volume_stats['min_air_pocket_depth'] = min(dct.get('depth') for dct in slice_stats.values())
    volume_stats['max_air_pocket_depth'] = max(dct.get('depth') for dct in slice_stats.values())


    if save:
        convert_slice_stats_to_csv(slice_stats,f'../data/aske/{dataset}/csv_files/slice_stats')
        convert_volume_stats_to_csv(volume_stats, f'../data/aske/{dataset}/csv_files/volume_stats')

    return slice_stats, volume_stats

## Start the analysis

In [4]:
dataset = 'WUP1'
path = DATA_PATH_TEMPLATE_1.format(dataset.lower())
DATA = initialize_dataset(path, dataset, width=1028, save=True)

In [5]:
DATA.shape

(1480, 1028, 1028)

In [6]:
import mlab

ModuleNotFoundError: No module named 'releases'

In [ ]:
thresh = adaptive_thresholding(DATA[0], block_size=201, constant_offset=50)
inverted_img = cv2.bitwise_not(thresh)

In [ ]:
# Specify the file names for saving
thresh_filename = 'thresh_image.png'
inverted_filename = 'inverted_image.png'



# Save the thresholded image
cv2.imwrite(thresh_filename, thresh)

# Save the inverted image
cv2.imwrite(inverted_filename, inverted_img)

In [ ]:
# Save the inverted image
cv2.imwrite('original.png', DATA[0])

In [ ]:
# run_analysis(DATA, dataset=dataset.lower(), save=True)

## Test to randomly apply the postprocessing to a series of images -> Used for visual inspection

In [ ]:
# import random

# # Assuming you want to select, for example, 5 random images from the list
# subset_size = 30

# # Randomly select indices for the subset
# subset_indices = random.sample(range(len(DATA)), subset_size)

# # Create a list of images from the selected indices
# random_subset = [(DATA[i], i) for i in subset_indices]

# for img, i in random_subset:
#     print(i)
#     adaptive_thresholding(img, block_size=201, constant_offset=50, plot=True)

## Save data as VTK

In [ ]:
def calculate_slices(memory_limit_gb, slice_shape=(1028, 1028), dtype=np.uint8):
    # Size of each element in bytes
    element_size = np.dtype(dtype).itemsize
    print(element_size)

    # Calculate the size of each slice in bytes
    slice_size_bytes = np.prod(slice_shape) * element_size
    print(slice_size_bytes)

    # Convert memory limit to bytes
    memory_limit_bytes = memory_limit_gb * 1024**3

    # Calculate the number of slices that can fit in memory
    num_slices = memory_limit_bytes // slice_size_bytes

    return num_slices


In [ ]:
# Example usage with a 4GB memory limit
memory_limit_gb = 4
slice_shape = (1028, 1028)

num_slices = calculate_slices(memory_limit_gb, slice_shape, dtype=DATA.dtype)
print(f"Number of slices that can fit in {memory_limit_gb}GB: {num_slices}")


In [ ]:
import vtk
import vtk.util.numpy_support as numpy_support

def numpyToVTK(data, output_file):
    data_type = vtk.VTK_FLOAT
    shape = data.shape

    flat_data_array = data.flatten()
    vtk_data = numpy_support.numpy_to_vtk(num_array=flat_data_array, deep=True, array_type=data_type)

    img = vtk.vtkImageData()
    img.GetPointData().SetScalars(vtk_data)
    img.SetDimensions(shape[0], shape[1], shape[2])

    # Save the VTK file
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(output_file)
    writer.SetInputData(img)
    writer.Write()

In [ ]:
output_file = 'test2.vti'  # Use '.vti' extension for XML-based VTK files
numpyToVTK(DATA[:200], output_file)

In [ ]:
def numpyToVTK_grayscale(volume_data, output_file):
    data_type = vtk.VTK_UNSIGNED_CHAR  # Use unsigned char for grayscale data
    num_components = 1  # Grayscale has one component per voxel
    shape = volume_data.shape

    # Flatten the volume data array
    flat_data_array = volume_data.flatten()
    vtk_data = numpy_support.numpy_to_vtk(num_array=flat_data_array, deep=True, array_type=data_type)

    img = vtk.vtkImageData()
    img.GetPointData().SetScalars(vtk_data)
    img.SetDimensions(shape[0], shape[1], shape[2])

    # Set the number of components for the grayscale data
    img.GetPointData().GetScalars().SetNumberOfComponents(num_components)

    # Save the VTK file
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(output_file)
    writer.SetInputData(img)
    writer.Write()

In [ ]:
output_file = 'test2.vti'  # Use '.vti' extension for XML-based VTK files
numpyToVTK_grayscale(DATA[:200], output_file)